# Big data lab

## import library

In [25]:
import pandas as pd
import pickle
import my_func
import lightgbm as lgb
from sklearn.model_selection import KFold 
file_name = 'churn_train_model_fe.pickle'
file_path = r'D:\BDL-Final\data-from-BDL' + '\\'

# settings for dataframe
pd.set_option('display.max_columns', None)

In [15]:
df_traine_fe = pd.read_pickle(file_path+file_name)

## Classification Metrics

In [27]:
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
def binary_classification_metrics(y_true_tr, y_pred_tr, y_true_val = None, y_pred_val = None, report = False):
    print ("{:<15} {:<10} {:<10} {:<10}".format('Metrics','Train','Test','\u0394'))
    metrics_dict = {}

    metrics_dict['roc_auc'] = np.round(roc_auc_score(y_true_tr, y_pred_tr),4)
    metrics_dict['accuracy'] = np.round(accuracy_score(y_true_tr, y_pred_tr),4)
    metrics_dict['precision'] = np.round(precision_score(y_true_tr, y_pred_tr),4)
    metrics_dict['recall'] = np.round(recall_score(y_true_tr, y_pred_tr),4)
    metrics_dict['f1_score'] = np.round(f1_score(y_true_tr, y_pred_tr),4)
    
    if y_true_val is not None:
        metrics_dict_test = {}
        metrics_dict_test['roc_auc'] = np.round(roc_auc_score(y_true_val, y_pred_val),4)
        metrics_dict_test['accuracy'] = np.round(accuracy_score(y_true_val, y_pred_val),4)
        metrics_dict_test['precision'] = np.round(precision_score(y_true_val, y_pred_val),4)
        metrics_dict_test['recall'] = np.round(recall_score(y_true_val, y_pred_val),4)
        metrics_dict_test['f1_score'] = np.round(f1_score(y_true_val, y_pred_val),4)
        # print(classification_report(y_true_val, y_pred_val))
    
        for metrics, value in metrics_dict.items():
            value_test = metrics_dict_test[metrics]
            diff = np.round(metrics_dict_test[metrics] - value, 4)
            print ("{:<15} {:<10} {:<10} {:<10}".format(metrics, value, value_test, diff))
    else:
        for metrics, value in metrics_dict.items():
            print ("{:<15} {:<10}".format(metrics, value))
    if report:
        print('\n')
        print('Train :')
        print(classification_report(y_true_tr, y_pred_tr))
        if not y_true_val.empty:
            print('Test :')
            print(classification_report(y_true_val, y_pred_val))

# Exploratory Data Analysis (EDA)

## First look

In [16]:
df_traine_fe.head(5)

,Ama_rchrgmnt_sum_max_mnt1,content_clc_mea_mnt1,content_cnt_max_mnt1,voice_out_short_part_max_mnt1,voice_mts_in_nrest_part_std_mnt1,num_act_days_max_mnt1,sms_roam_clc_min_mnt1,voice_in_cmpttrs_avg_durmin_mnt1,com_num_part_mea_mnt1,pay_avg_mea_mnt1,voice_out_tar_dur_std_mnt1,voice_out_tar_dur_min_mnt1,gprs_clc_mea_mnt1,pay_max_min_mnt3,voice_in_short_part_mea_mnt3,voice_in_roam_clc_std_mnt1,conn_out_uniq_cnt_std_mnt1,voice_mts_in_dwork_part_td_mnt3,data_3g_tar_vol_mea_mnt3,voice_in_fix_tar_dur_mea_mnt3,voice_in_tar_dur_max_mnt3,ks_num_part_td_mnt3,data_3g_dou_mea_mnt3,voice_mts_out_drest_partmea_mnt1,conn_com_part_min_mnt1,Ama_rchrgmnt_sum_mea_mnt3,voice_mts_in_dwork_part_mea_wk1,voice_mts_in_dwork_part_std_mnt1,gprs_clc_std_mnt1,conn_com_part_td_mnt3,sms_clc_min_mnt3,gprs_clc_mea_mnt3,gprs_clc_min_mnt3,voice_in_cmpttrs_avg_durmax_mnt3,voice_in_tar_dur_td_mnt3,conn_in_uniq_cnt_td_mnt3,voice_mts_out_nrest_partmax_mnt1,voice_in_tar_dur_std_mnt1,voice_mts_out_dwork_partmea_wk1,voice_out_fix_tar_dur_td_mnt3,all_clc_mea_mnt1,voice_out_cmpttrs_td_cnttd_mnt3,all_roam_clc_mea_mnt1,voice_in_tar_dur_std_mnt3,voice_in_td_cnt_max_mnt3,voice_out_fix_tar_dur_mea_wk1,conn_com_part_std_mnt3,voice_in_roam_clc_mea_mnt3,pay_p2p_in_sum_max_mnt3,content_cnt_mea_mnt1,voice_in_roam_clc_mea_wk1,com_num_part_min_mnt1,pay_p2p_in_sum_mea_wk1,all_home_clc_td_mnt3,voice_out_td_cnt_mea_mnt1,vas_clc_mea_wk1,com_num_part_max_mnt1,voice_mts_out_drest_partstd_mnt3,pay_sum_td_mnt3,pay_p2p_in_sum_std_mnt1,conn_out_uniq_cnt_std_mnt3,all_clc_min_mnt3,all_roam_clc_min_mnt3,pay_p2p_out_sum_max_mnt3,voice_in_cmpttrs_td_cnt_max_mnt1,voice_mts_out_nrest_partstd_mnt3,voice_in_td_cnt_mea_mnt3,data_3g_dou_min_mnt3,voice_mts_in_nrest_part_mea_wk1,voice_in_fix_tar_dur_max_mnt1,ks_num_part_mea_wk1,pay_p2p_out_sum_std_mnt1,block_all_dur_min_mnt1,voice_in_cmpttrs_td_cnt_max_mnt3,pay_sum_std_mnt1,abon_part_max_mnt3,sms_out_cnt_std_mnt3,sms_in_cnt_min_mnt1,conn_in_uniq_cnt_mea_wk1,pay_sum_min_mnt3,ama_volume_max_mnt3,vas_clc_min_mnt3,voice_in_td_cnt_std_mnt1,sms_roam_clc_max_mnt1,all_cnt_min_mnt1,voice_in_short_part_max_mnt1,accum_mts_dur_std_mnt3,data_3g_tv_cnt_mea_wk1,voice_out_td_cnt_min_mnt1,data_3g_tar_vol_mea_mnt1,voice_mts_out_nrest_partmin_mnt1,all_cnt_mea_mnt1,data_3g_dou_std_mnt3,voice_in_short_part_min_mnt3,sms_in_cnt_td_mnt3,voice_mts_out_dwork_partstd_mnt1,voice_out_short_part_mea_wk1,voice_in_fix_tar_dur_mea_wk1,block_all_dur_min_mnt3,pay_sum_max_mnt1,content_cnt_mea_wk1,voice_mts_out_nwork_partstd_mnt1,com_num_part_mea_mnt3,conn_out_uniq_cnt_max_mnt3,content_clc_mea_mnt3,Ama_rchrgmnt_sum_min_mnt3,abon_part_min_mnt3,conn_out_uniq_cnt_mea_mnt3,gprs_clc_std_mnt3,voice_out_short_part_min_mnt1,all_cnt_std_mnt1,conn_com_part_mea_wk1,com_num_part_max_mnt3,voice_out_fix_tar_dur_max_mnt1,pay_p2p_out_sum_mea_mnt1,abon_part_min_mnt1,accum_mts_dur_min_mnt1,content_clc_min_mnt1,voice_out_fix_tar_dur_std_mnt1,voice_mts_out_nwork_partmin_mnt1,vas_clc_max_mnt3,sms_roam_clc_mea_wk1,pay_avg_std_mnt1,content_clc_min_mnt3,pay_p2p_out_sum_mea_mnt3,data_3g_tar_vol_std_mnt1,vas_clc_min_mnt1,sms_in_cnt_mea_mnt1,goodok_clc_max_mnt3,com_num_part_td_mnt3,all_home_clc_mea_mnt3,voice_in_short_part_max_mnt3,pay_max_max_mnt1,sms_clc_std_mnt1,pay_max_mea_mnt1,accum_mts_dur_td_mnt3,gprs_clc_max_mnt3,vas_clc_mea_mnt1,all_clc_mea_mnt3,voice_in_kievstar_part_min_mnt1,Ama_rchrgmnt_sum_mea_wk1,sms_in_cnt_min_mnt3,voice_mts_in_drest_part_td_mnt3,voice_mts_in_dwork_part_min_mnt1,voice_in_cmpttrs_td_cnt_mea_mnt1,sms_roam_clc_max_mnt3,pay_max_mea_mnt3,gprs_clc_td_mnt3,ks_num_part_std_mnt1,voice_in_mts_avg_dur_max_mnt1,voice_in_td_cnt_mea_mnt1,voice_mts_in_drest_part_max_mnt3,accum_mts_dur_max_mnt3,pay_p2p_out_sum_max_mnt1,sms_roam_clc_std_mnt3,voice_out_cmpttrs_td_cntmin_mnt3,voice_out_cmpttrs_avg_dumax_mnt1,voice_mts_in_nrest_part_mea_mnt3,voice_out_tar_dur_max_mnt1,voice_mts_in_drest_part_std_mnt1,clc_no_vas_roam_std_mnt3,non_accum_internet_vol_mea_wk1,voice_mts_out_nwork_partmea_mnt1,voice_mts_in_nrest_

In [18]:
df_traine_fe.describe()

,Ama_rchrgmnt_sum_max_mnt1,content_clc_mea_mnt1,content_cnt_max_mnt1,voice_out_short_part_max_mnt1,voice_mts_in_nrest_part_std_mnt1,num_act_days_max_mnt1,sms_roam_clc_min_mnt1,voice_in_cmpttrs_avg_durmin_mnt1,com_num_part_mea_mnt1,pay_avg_mea_mnt1,voice_out_tar_dur_std_mnt1,voice_out_tar_dur_min_mnt1,gprs_clc_mea_mnt1,pay_max_min_mnt3,voice_in_short_part_mea_mnt3,voice_in_roam_clc_std_mnt1,conn_out_uniq_cnt_std_mnt1,voice_mts_in_dwork_part_td_mnt3,data_3g_tar_vol_mea_mnt3,voice_in_fix_tar_dur_mea_mnt3,voice_in_tar_dur_max_mnt3,ks_num_part_td_mnt3,data_3g_dou_mea_mnt3,voice_mts_out_drest_partmea_mnt1,conn_com_part_min_mnt1,Ama_rchrgmnt_sum_mea_mnt3,voice_mts_in_dwork_part_mea_wk1,voice_mts_in_dwork_part_std_mnt1,gprs_clc_std_mnt1,conn_com_part_td_mnt3,sms_clc_min_mnt3,gprs_clc_mea_mnt3,gprs_clc_min_mnt3,voice_in_cmpttrs_avg_durmax_mnt3,voice_in_tar_dur_td_mnt3,conn_in_uniq_cnt_td_mnt3,voice_mts_out_nrest_partmax_mnt1,voice_in_tar_dur_std_mnt1,voice_mts_out_dwork_partmea_wk1,voice_out_fix_tar_dur_td_mnt3,all_clc_mea_mnt1,voice_out_cmpttrs_td_cnttd_mnt3,all_roam_clc_mea_mnt1,voice_in_tar_dur_std_mnt3,voice_in_td_cnt_max_mnt3,voice_out_fix_tar_dur_mea_wk1,conn_com_part_std_mnt3,voice_in_roam_clc_mea_mnt3,pay_p2p_in_sum_max_mnt3,content_cnt_mea_mnt1,voice_in_roam_clc_mea_wk1,com_num_part_min_mnt1,pay_p2p_in_sum_mea_wk1,all_home_clc_td_mnt3,voice_out_td_cnt_mea_mnt1,vas_clc_mea_wk1,com_num_part_max_mnt1,voice_mts_out_drest_partstd_mnt3,pay_sum_td_mnt3,pay_p2p_in_sum_std_mnt1,conn_out_uniq_cnt_std_mnt3,all_clc_min_mnt3,all_roam_clc_min_mnt3,pay_p2p_out_sum_max_mnt3,voice_in_cmpttrs_td_cnt_max_mnt1,voice_mts_out_nrest_partstd_mnt3,voice_in_td_cnt_mea_mnt3,data_3g_dou_min_mnt3,voice_mts_in_nrest_part_mea_wk1,voice_in_fix_tar_dur_max_mnt1,ks_num_part_mea_wk1,pay_p2p_out_sum_std_mnt1,block_all_dur_min_mnt1,voice_in_cmpttrs_td_cnt_max_mnt3,pay_sum_std_mnt1,abon_part_max_mnt3,sms_out_cnt_std_mnt3,sms_in_cnt_min_mnt1,conn_in_uniq_cnt_mea_wk1,pay_sum_min_mnt3,ama_volume_max_mnt3,vas_clc_min_mnt3,voice_in_td_cnt_std_mnt1,sms_roam_clc_max_mnt1,all_cnt_min_mnt1,voice_in_short_part_max_mnt1,accum_mts_dur_std_mnt3,data_3g_tv_cnt_mea_wk1,voice_out_td_cnt_min_mnt1,data_3g_tar_vol_mea_mnt1,voice_mts_out_nrest_partmin_mnt1,all_cnt_mea_mnt1,data_3g_dou_std_mnt3,voice_in_short_part_min_mnt3,sms_in_cnt_td_mnt3,voice_mts_out_dwork_partstd_mnt1,voice_out_short_part_mea_wk1,voice_in_fix_tar_dur_mea_wk1,block_all_dur_min_mnt3,pay_sum_max_mnt1,content_cnt_mea_wk1,voice_mts_out_nwork_partstd_mnt1,com_num_part_mea_mnt3,conn_out_uniq_cnt_max_mnt3,content_clc_mea_mnt3,Ama_rchrgmnt_sum_min_mnt3,abon_part_min_mnt3,conn_out_uniq_cnt_mea_mnt3,gprs_clc_std_mnt3,voice_out_short_part_min_mnt1,all_cnt_std_mnt1,conn_com_part_mea_wk1,com_num_part_max_mnt3,voice_out_fix_tar_dur_max_mnt1,pay_p2p_out_sum_mea_mnt1,abon_part_min_mnt1,accum_mts_dur_min_mnt1,content_clc_min_mnt1,voice_out_fix_tar_dur_std_mnt1,voice_mts_out_nwork_partmin_mnt1,vas_clc_max_mnt3,sms_roam_clc_mea_wk1,pay_avg_std_mnt1,content_clc_min_mnt3,pay_p2p_out_sum_mea_mnt3,data_3g_tar_vol_std_mnt1,vas_clc_min_mnt1,sms_in_cnt_mea_mnt1,goodok_clc_max_mnt3,com_num_part_td_mnt3,all_home_clc_mea_mnt3,voice_in_short_part_max_mnt3,pay_max_max_mnt1,sms_clc_std_mnt1,pay_max_mea_mnt1,accum_mts_dur_td_mnt3,gprs_clc_max_mnt3,vas_clc_mea_mnt1,all_clc_mea_mnt3,voice_in_kievstar_part_min_mnt1,Ama_rchrgmnt_sum_mea_wk1,sms_in_cnt_min_mnt3,voice_mts_in_drest_part_td_mnt3,voice_mts_in_dwork_part_min_mnt1,voice_in_cmpttrs_td_cnt_mea_mnt1,sms_roam_clc_max_mnt3,pay_max_mea_mnt3,gprs_clc_td_mnt3,ks_num_part_std_mnt1,voice_in_mts_avg_dur_max_mnt1,voice_in_td_cnt_mea_mnt1,voice_mts_in_drest_part_max_mnt3,accum_mts_dur_max_mnt3,pay_p2p_out_sum_max_mnt1,sms_roam_clc_std_mnt3,voice_out_cmpttrs_td_cntmin_mnt3,voice_out_cmpttrs_avg_dumax_mnt1,voice_mts_in_nrest_part_mea_mnt3,voice_out_tar_dur_max_mnt1,voice_mts_in_drest_part_std_mnt1,clc_no_vas_roam_std_mnt3,non_accum_internet_vol_mea_wk1,voice_mts_out_nwork_partmea_mnt1,voice_mts_in_nrest_

In [19]:
# перевірка на категоріальні значення
cat_info = my_func.categorical_col_info(df_traine_fe)

In [20]:
cat_info

,Column,Unique values,Missing values
Total,0,0,0


# Feature Engineering


In [21]:
# перевірка на missing значення
ms_val = my_func.count_missing_values(df_traine_fe)

In [22]:
ms_val

,Column,Missing Values,Missing Values (%)
709,bs_succ_rate,150000,100.000000
708,bs_drop_call_rate,150000,100.000000
645,device_has_gprs,150000,100.000000
716,bs_of_succ_but_drop_m1,150000,100.000000
715,bs_of_unsucc_attemp_equip_m1,150000,100.000000
...,...,...,...
632,loc_market_share,880,0.586667
631,loc_cnt_events,880,0.586667
630,loc_is_obl_center,880,0.586667
606,lt,673,0.448667


In [23]:


# Filter the DataFrame to select columns with 100% missing values
df_missing_100 = ms_val[ms_val['Missing Values (%)'] == 100.0]

# Print the resulting DataFrame
df_missing_100.describe()


,Missing Values,Missing Values (%)
count,11.0,11.0
mean,150000.0,100.0
std,0.0,0.0
min,150000.0,100.0
25%,150000.0,100.0
50%,150000.0,100.0
75%,150000.0,100.0
max,150000.0,100.0


In [24]:
# prepare dataset for training
def ds_prep(df):
    # missing change for -1
    df.fillna(-1, inplace=True)

    # remove abon_id

    df = df.drop('abon_id', axis=1)

9580.0

In [ ]:
model_pipeline = {'Model_name': 'Titanic',
                  'preprocess': preprocess_string,
                  'algorythm': 'dont know yet',
                  'model': 'in process',
                  'score': 0.9}
                 # 'features': df_train.columns}

In [ ]:
y = df_traine_fe.target
X = df_traine_fe.drop(['target', 'abon_id'], axis=1)

In [28]:
def my_trainer_lgb(model, X_train):
    kf = KFold(n_splits=5, shuffle=True, random_state=777)
    callback2 = lgb.log_evaluation(period = 50)
    list_tr_acc = []
    list_val_acc = []
    n=0
    for train_index, val_index in kf.split(X_train):
        n +=1
        # print('Fold№', n)
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        model.fit(X_tr, y_tr, eval_set=[(X_tr, y_tr),(X_val, y_val)], eval_names=['Train', 'Val'], early_stopping_rounds=20, eval_metric=['binary_error'], callbacks=[callback2])
        
        y_tr_pred = model.predict(X_tr)
        y_val_pred = model.predict(X_val)
        
        list_tr_acc.append(accuracy_score(y_tr, y_tr_pred))
        list_val_acc.append(accuracy_score(y_val, y_val_pred))

    print('total Accuracy train= ', np.mean(np.array(list_tr_acc)))
    print('total Accuracy val= ', np.mean(np.array(list_val_acc)))
    return np.mean(np.array(list_val_acc))  - (np.mean(np.array(list_tr_acc)) - np.mean(np.array(list_val_acc)))

In [ ]:
from hyperopt import hp, fmin, tpe
import numpy as np
from sklearn.metrics import auc
from sklearn.model_selection import cross_val_score, KFold

In [29]:
clf = lgb.LGBMClassifier(**best)

NameError: name 'best' is not defined